In [19]:
%load_ext autoreload
%autoreload 2

from sae_lens import SAE
import os
import numpy as np

os.environ["GEMMA_2_SAE_WEIGHTS_ROOT"] = "/workspace/weights/"
assert os.path.exists(os.environ["GEMMA_2_SAE_WEIGHTS_ROOT"])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from huggingface_hub import hf_hub_download

path_to_params = hf_hub_download(
    repo_id="google/gemma-scope-2b-pt-res",
    filename=".",
    local_dir='/workspace/weights/release-preview/gemmascope-2b-pt-res',
    force_download=False,
)

EntryNotFoundError: 404 Client Error. (Request ID: Root=1-66aad802-30e9a10a6887dc877c3e9969;aa887b1a-11b0-447b-b0f5-bc4c532ff3b2)

Entry Not Found for url: https://huggingface.co/google/gemma-scope-2b-pt-res/resolve/main/.
Invalid blob

In [26]:

from huggingface_hub import HfFileSystem

def get_gemma2_2b_SAE_path(layer, width=16, closest_l0=100):
    fs = HfFileSystem()
    all_paths = fs.glob("google/gemma-scope-2b-pt-res/**/params.npz")
    
    candidate_paths = [p for p in all_paths if f'layer_{layer}/width_{width}k/average_l0_' in p]
    
    # get the l0 value from the path
    l0_values = [int(p.split('average_l0_')[1].split('/')[0]) for p in candidate_paths]
    # print(l0_values)
    # print(*candidate_paths, sep='\n')
    
    # find the one closest to closest_l0
    idx = np.argmin(np.abs(np.array(l0_values) - closest_l0))
    desire_l0 = l0_values[idx]
    desire_path = candidate_paths[idx]
    
    return desire_l0, desire_path



for layer in range(12):
    l0, path = get_gemma2_2b_SAE_path(layer)
    print(layer, l0, path)


    

0 105 google/gemma-scope-2b-pt-res/layer_0/width_16k/average_l0_105/params.npz
1 102 google/gemma-scope-2b-pt-res/layer_1/width_16k/average_l0_102/params.npz
2 141 google/gemma-scope-2b-pt-res/layer_2/width_16k/average_l0_141/params.npz
3 59 google/gemma-scope-2b-pt-res/layer_3/width_16k/average_l0_59/params.npz
4 124 google/gemma-scope-2b-pt-res/layer_4/width_16k/average_l0_124/params.npz
5 68 google/gemma-scope-2b-pt-res/layer_5/width_16k/average_l0_68/params.npz
6 70 google/gemma-scope-2b-pt-res/layer_6/width_16k/average_l0_70/params.npz
7 69 google/gemma-scope-2b-pt-res/layer_7/width_16k/average_l0_69/params.npz
8 71 google/gemma-scope-2b-pt-res/layer_8/width_16k/average_l0_71/params.npz
9 73 google/gemma-scope-2b-pt-res/layer_9/width_16k/average_l0_73/params.npz
10 77 google/gemma-scope-2b-pt-res/layer_10/width_16k/average_l0_77/params.npz
11 80 google/gemma-scope-2b-pt-res/layer_11/width_16k/average_l0_80/params.npz


In [10]:
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release="gemmascope-2b-pt-res",
    sae_id="layer_20/width_16k/average_l0_71",
    device='cuda'
)

hi
gemma-2-saes
release-preview/gemmascope-2b-pt-res/layer_20/width_16k/average_l0_71/params.npz
gemmascope_bucket


In [11]:
sae

SAE(
  (activation_fn): ReLU()
  (hook_sae_input): HookPoint()
  (hook_sae_acts_pre): HookPoint()
  (hook_sae_acts_post): HookPoint()
  (hook_sae_output): HookPoint()
  (hook_sae_recons): HookPoint()
  (hook_sae_error): HookPoint()
)